In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import pandas as pd
from tqdm.notebook import tqdm

/home/letfd/virtual_envs/base_env/lib/python3.8/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (None)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(


In [2]:
model = SentenceTransformer(
    "distiluse-base-multilingual-cased-v1", device="cuda"
)  # or device="cpu" if you don't have a GPU

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [3]:
df = pd.read_csv("../data/yappy_hackaton_2024_400k.csv")
print(df.shape)
df.head(2)

(400000, 2)


,link,description
0,https://cdn-st.rutubelist.ru/media/b0/e9/ef285...,"#нарезкистримов , #dota2 , #cs2 , #fifa23 , #m..."
1,https://cdn-st.rutubelist.ru/media/39/6c/b31bc...,🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #rob...


In [4]:
df.isnull().sum(0)

link               0
description    54919
dtype: int64

In [5]:
df.dropna(inplace=True)
print(df.shape)

(345081, 2)


In [6]:
%%time

vectors = model.encode(
    # [row.description for row in df.itertuples()],
    df.description.tolist(),
    show_progress_bar=True,
)

Batches:   0%|          | 0/10784 [00:00<?, ?it/s]

CPU times: user 3min 33s, sys: 11.4 s, total: 3min 44s
Wall time: 2min 34s


In [7]:
np.save("../data/lct_description_vectors.npy", vectors, allow_pickle=False)

In [10]:
# Import client library
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

client = QdrantClient("http://localhost:6333")

In [11]:
client.recreate_collection(
    collection_name="lct_description",
    vectors_config=VectorParams(size=512, distance=Distance.COSINE),
)

/tmp/ipykernel_20266/1533277188.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [12]:
%%time

pl = iter([row.to_dict() for _, row in df.iterrows()])

CPU times: user 22.2 s, sys: 180 ms, total: 22.4 s
Wall time: 22.3 s


In [15]:
%%time

client.upload_collection(
    collection_name="lct_description",
    vectors=vectors,
    payload=pl,
    ids=None,  # Vector ids will be assigned automatically
    batch_size=256,  # How many vectors will be uploaded in a single request?
)

CPU times: user 15min 53s, sys: 8.98 s, total: 16min 2s
Wall time: 16min 32s


In [16]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer


class NeuralSearcher:
    def __init__(self, collection_name):
        self.collection_name = collection_name
        # Initialize encoder model
        self.model = SentenceTransformer("distiluse-base-multilingual-cased-v1", device="cuda")
        # initialize Qdrant client
        self.qdrant_client = QdrantClient("http://localhost:6333")
        
    def search(self, text: str):
        # Convert text query into vector
        vector = self.model.encode(text).tolist()

        # Use `vector` for search for closest vectors in the collection
        search_result = self.qdrant_client.search(
            collection_name=self.collection_name,
            query_vector=vector,
            query_filter=None,  # If you don't want any filters for now
            limit=10,  # 5 the most closest results is enough
        )
        # `search_result` contains found vector ids with similarity scores along with the stored payload
        # In this function you are interested in payload only
        payloads = [hit.payload for hit in search_result]
        return payloads

In [17]:
neural_searcher = NeuralSearcher(collection_name="lct_description")


In [62]:
%%time

neural_searcher.search(text='кукинг')

CPU times: user 17.2 ms, sys: 8.11 ms, total: 25.3 ms
Wall time: 25.7 ms


[{'description': 'Кукуруза',
  'link': 'https://cdn-st.rutubelist.ru/media/0e/a6/2fc37e6c47f7807c5f967f4fef68/fhd.mp4'},
 {'description': 'гача',
  'link': 'https://cdn-st.rutubelist.ru/media/64/95/aedf9df84e0dab31c7aa21685127/fhd.mp4'},
 {'description': 'гача',
  'link': 'https://cdn-st.rutubelist.ru/media/a3/f9/b56a51b44b2780814a69baad88a4/fhd.mp4'},
 {'description': 'Кабачок',
  'link': 'https://cdn-st.rutubelist.ru/media/e2/3d/f627156d4d599a08e12d9cc3b813/fhd.mp4'},
 {'description': 'Канеки',
  'link': 'https://cdn-st.rutubelist.ru/media/ee/aa/0b7a879b4db6817bc9b2ecbcbe33/fhd.mp4'},
 {'description': 'devochkki',
  'link': 'https://cdn-st.rutubelist.ru/media/60/e2/3e7ec1ad4b22a0f7d2f35818b037/fhd.mp4'},
 {'description': 'Крупа',
  'link': 'https://cdn-st.rutubelist.ru/media/b9/ee/300ec4ff496082d86a8c3303bc6b/fhd.mp4'},
 {'description': 'cozyyyy 🧸🧸',
  'link': 'https://cdn-st.rutubelist.ru/media/a2/73/f9ab48d54bb9b128ef49b5eeddb5/fhd.mp4'},
 {'description': 'сушка',
  'link': 'https:

In [19]:
%%time

import requests
resp = requests.get("http://127.0.0.1:8000/search", params={'query': 'домашние животные'})

CPU times: user 6 ms, sys: 623 µs, total: 6.62 ms
Wall time: 328 ms


In [20]:
resp.json()

[{'description': '#животные',
  'link': 'https://cdn-st.rutubelist.ru/media/d7/7d/203e10a2439caec9e36bc3bcc132/fhd.mp4'},
 {'description': '#питомцы',
  'link': 'https://cdn-st.rutubelist.ru/media/33/75/9db45eeb4f639c94a6425346e487/fhd.mp4'},
 {'description': '#питомцы',
  'link': 'https://cdn-st.rutubelist.ru/media/91/67/c47e99304049b04d355900b6109c/fhd.mp4'},
 {'description': 'Животные в прошлом',
  'link': 'https://cdn-st.rutubelist.ru/media/27/9f/876097984fd8a713675f1e38670e/fhd.mp4'},
 {'description': 'Ваше любимое животное',
  'link': 'https://cdn-st.rutubelist.ru/media/1d/99/f31b870c4d8ea9e16a08d41f5268/fhd.mp4'},
 {'description': 'А у вас есть домашнее животное ❓',
  'link': 'https://cdn-st.rutubelist.ru/media/80/2a/412e15764e259dd53517c9345150/fhd.mp4'},
 {'description': '#чихуахуа#животные',
  'link': 'https://cdn-st.rutubelist.ru/media/75/aa/0513064a423aa5234d8101b2f169/fhd.mp4'},
 {'description': '#кот #питомцы #животные',
  'link': 'https://cdn-st.rutubelist.ru/media/de/e1